### Problem Four

[lab7:prob:accurate]:(#Problem-Four)

Which grid gives the best accuracy for $d=R/2$?
Explain in what ways it is more accurate.  Consider the accuracy of the frequency, but also the accuracy of the group speed (the gradient of the frequency with respect to wavenumber).  Describe what ranges of wavenumber the accuracy is good and what ranges it is less good.


For grid 1
$$\omega ^2 = f^2 \cos^2(\frac{kd}{2}) + \frac{4gH \sin^2(\frac{kd}{2})}{d^2}$$

$$\omega ^2 = f^2 \cos^2(\frac{kR}{4}) + \frac{4gH \sin^2(\frac{kR}{4})}{(R/2)^2}$$

Show that for grid 2
$$\omega ^2 = f^2  + \frac{4gH \sin^2(\frac{kd}{2})}{d^2}$$
$$\omega ^2 = f^2  + \frac{4gH \sin^2(\frac{kR}{4})}{(R/2)^2}$$

### Problem Five

[lab7:prob:noy]:(#Problem-Five)

Modify *rain.py* to solve equations
([No variation in y, first eqn](#lab7:sec:firsteq)), ([No variation in y, second eqn](#lab7:sec:secondeq)) and ([No variation in y, third eqn](#lab7:sec:thirdeq)) on the most accurate
grid.


<div id='lab7:sec:firsteq'>
(No variation in y, first eqn)
$$\frac{\partial u}{\partial t} - fv = -g\frac{\partial h}{\partial x}$$ 
</div><div id='lab7:sec:secondeq'>
(No variation in y, second eqn)
$$\frac{\partial v}{\partial t} + fu = 0$$
</div><div id='lab7:sec:thirdeq'>
(No variation in y, third eqn)
$$\frac{\partial h}{\partial t} + H\frac{\partial u}{\partial x} = 0$$
</div>

In [ ]:
from numlabs.lab7 import rain
# Run 5 time steps on a 9 point grid
rain.rain(5,9)